In [ ]:
from coreLib.dataset import DataSet
data_dir       = "/media/ansary/DriveData/Work/bengalAI/datasets/CraftData/source"
ds=DataSet(data_dir)
ds.common.symbols.df

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import numpy as np
iden=2
from coreLib.word import create_word
'''
source_type             :       bangla/english 
data_type               :       handwritten/printed                  
comp_type               :       grapheme/number
'''
for source in ["bangla","english"]:
    for data in ["handwritten","printed"]:
        for comp in ["grapheme","number","mixed"]:
            for ud in [True,False]:
                
                print("=========================================")
                print(source,data,comp,str(ud))
                img,label,iden=create_word(iden,
                                source,
                                data,
                                comp,
                                ds,
                                use_dict=ud)
                plt.imshow(img)
                plt.show()
                print(label)
                print(np.unique(img))
                print("=========================================")


In [ ]:
print("ত"+"্"+'\u200d')


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import numpy as np
iden=2
from coreLib.word import create_word
imgs=[]
labels=[]
w_max=0
h_max=0
iden = 3
for _ in range(3):
    img,label,iden=create_word(iden,
                                "bangla",
                                "printed",
                                "grapheme",
                                ds,
                                use_dict=True)
    # max neutralization
    plt.imshow(img)
    plt.show()
    vals=list(np.unique(img))
    vals=sorted(vals,reverse=True)
    vals=vals[:-1]
    _img=np.zeros(img.shape)
    for v,l in zip(vals,label.keys()):
        _img[img==v]=l
    plt.imshow(_img)
    plt.show()
    
    labels.append(label)
    imgs.append(_img)
    print(label)

In [ ]:
back=np.zeros((120,30))
plt.imshow(back)
plt.show()

# Problem-1: ৩ টার ভ্যালু মার্জ হয়ে যাইতেছে  (dashed charecters are being created)

In [ ]:
img=imgs[0]
print(labels[0])
back=np.zeros(img.shape)
back[img==4]=255
img1=img.copy()
# co-ordinate
idx = np.where(img==4)
y_min,y_max,x_min,x_max = np.min(idx[0]), np.max(idx[0]), np.min(idx[1]), np.max(idx[1])
x1,x2,x3,x4=x_min,x_max,x_max,x_min
y1,y2,y3,y4=y_min,y_min,y_max,y_max
coords=np.array([[x1,y1],[x2,y2],[x3,y3],[x4,y4]],dtype="float32")

img1 = cv2.rectangle(img1, (x1,y1), (x3,y3), 5, 2)
plt.imshow(img1)
plt.plot(x1, y1, 'go--', linewidth=2, markersize=12)
plt.plot(x2, y2, 'go--', linewidth=2, markersize=12)
plt.plot(x3, y3, 'go--', linewidth=2, markersize=12)
plt.plot(x4, y4, 'go--', linewidth=2, markersize=12)

plt.show()

plt.imshow(back)
plt.show()
plt.imshow(img)
plt.show()

# get shape
(h, w) = img.shape
# get center
center = (w / 2, h / 2)
# angle to rotate
angle  = 30
# get rotation matrix
M = cv2.getRotationMatrix2D(center, angle, 1.0)
# Now will take out sin and cos values from rotationMatrix
# Also used numpy absolute function to make positive value
cosM = np.abs(M[0][0])
sinM = np.abs(M[0][1])
# Now will compute new height & width of
# an image so that we can use it in
# warpAffine function to prevent cropping of image sides
nH = int((h * sinM) +(w * cosM))
nW = int((h * cosM) +(w * sinM))

# After computing the new height & width of an image
# we also need to update the values of rotation matrix
M[0][2] += (nW/2) - center[0]
M[1][2] += (nH/2) - center[1]
# rotate
img = cv2.warpAffine(img, M, (nH,nW))


new_coords=[]
coord_mat=np.concatenate([coords,np.ones((4,1))],axis=1)
for c in coord_mat:
    new_coords.append(np.dot(M,c))
print(new_coords)
for c in new_coords:
    plt.plot(c[0], c[1], 'go--', linewidth=2, markersize=12)

plt.imshow(img)
plt.show()

img1=img.copy()
# co-ordinate
idx = np.where(img==4)
y_min,y_max,x_min,x_max = np.min(idx[0]), np.max(idx[0]), np.min(idx[1]), np.max(idx[1])
x1,x2,x3,x4=x_min,x_max,x_max,x_min
y1,y2,y3,y4=y_min,y_min,y_max,y_max
print(np.array([[x1,y1],[x2,y2],[x3,y3],[x4,y4]],dtype="float32"))

img1 = cv2.rectangle(img1, (x1,y1), (x3,y3), 5, 2)
plt.imshow(img1)
plt.plot(x1, y1, 'go--', linewidth=2, markersize=12)
plt.plot(x2, y2, 'go--', linewidth=2, markersize=12)
plt.plot(x3, y3, 'go--', linewidth=2, markersize=12)
plt.plot(x4, y4, 'go--', linewidth=2, markersize=12)

plt.show()



In [ ]:
back=np.zeros(img.shape)
back[img==4]=255
plt.imshow(back)
plt.show()

# Problem-2: After merging various rotational images , we loose center (rotation also)

In [ ]:
imgs=[]
labels=[]
w_max=0
h_max=0
iden = 3
for _ in range(3):
    img,label,iden=create_word(iden,
                                "bangla",
                                "handwritten",
                                "grapheme",
                                ds,
                                use_dict=True)
    plt.imshow(img)
    plt.show()
    h,w=img.shape
    if w>w_max:
        w_max=w
    if h>h_max:
        h_max=h
    print(label)
    imgs.append(img)
    labels.append(label)

In [ ]:
img=imgs[0]
# get shape
(h, w) = img.shape
# get center
center = (w / 2, h / 2)
# angle to rotate
angle  = 30
# get rotation matrix
M = cv2.getRotationMatrix2D(center, angle, 1.0)
# Now will take out sin and cos values from rotationMatrix
# Also used numpy absolute function to make positive value
cosM = np.abs(M[0][0])
sinM = np.abs(M[0][1])
# Now will compute new height & width of
# an image so that we can use it in
# warpAffine function to prevent cropping of image sides
nH = int((h * sinM) +(w * cosM))
nW = int((h * cosM) +(w * sinM))

# After computing the new height & width of an image
# we also need to update the values of rotation matrix
M[0][2] += (nW/2) - center[0]
M[1][2] += (nH/2) - center[1]
# rotate
img = cv2.warpAffine(img, M, (nH,nW))
plt.imshow(img)
plt.show()

In [ ]:
imgs=[img,imgs[2],imgs[1]]


In [ ]:
def padImage(img):
    '''
        pads an image with white value
        args:
            img     :       the image to pad
            pad_loc :       (lr/tb) lr: left-right pad , tb=top_bottom pad
            pad_dim :       the dimension to pad upto 
    '''    
    pad_dim=2048
    # shape
    h,w=img.shape
    # pad widths
    left_pad_width =(pad_dim-w)//2
    # print(left_pad_width)
    right_pad_width=pad_dim-w-left_pad_width
    # pads
    left_pad =np.zeros((h,left_pad_width))
    right_pad=np.zeros((h,right_pad_width))
    # pad
    img =np.concatenate([left_pad,img,right_pad],axis=1)
    return img

padded=[]
for img in imgs:
    padded.append(padImage(img))
ii=np.concatenate(padded,axis=0)
plt.imshow(ii)
plt.show()

In [ ]:
back=np.zeros(ii.shape)
back[ii==4]=255
plt.imshow(back)
plt.show()

# Problem-1: Value Does not stay constant for labels (dashed charecters are being created)
# Problem-2: After merging various rotational images , we loose center (rotation also)